<a href="https://colab.research.google.com/github/jpo85/meshroom_colab/blob/main/runtime_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fotogrammetria tramite Google Cloud & Meshroom

Questa pagina web si prefigge lo scopo di facilitare l'elaborazione di un dataset di foto per tentare una prima elaborazione e ricostruzione fotogrammetrica. Gli strumenti che vengono utilizzati per eseguire l'elaborazione sono il servizio Google colab e Meshroom.

Il servizio Google Colab, ha lo scopo di eseguire in cloud lo script composto da comandi shell bash e python per l'elaborazione delle immagini e la creazione della mesh 3d finale.
Google Colab è una piattaforma di google, nata principalmente per eseguire quelle operazioni di machine learning che richiedono prestazioni e potenza di calcolo elevata. Nel nostro caso specifico verrà utilizzato per fini alternativi, e verrà messo a disposizione dall'infrastruttura cloud la seguente potenza di calcolo:

*   32 GB di RAM
*   GPU con 16GB di RAM dedicata e funzionalità CUDA
*   Circa 40GB su filesystem

Meshroom è un software con licenza opensource che consente tramite foto e video di effettuare una ricostruzione fotogrammetrica, basandosi sul framework AliceVision. L'interfaccia di utilizzo è basata sulla logica dei workflow, in cui ogni blocco è collegato al successivo salvando su una folder l'elaborazione temporanea ed infine generando una mesh poligonale. Questo runtime consente eventualmente anche di utilizzare un workflow custom in cui è possibile avere più risultati diversi:
si può ad esempio aggiungere un workflow di post produzione che effettua un Meshdecimate + Retexturing + Retopology della mesh, oppure esportare la pointcloud prima che venga creata la mesh;
si può aggiungere anche un workflow che esporta la mesh finale ed effettua l'upload su sketchfab.

Per fare tutto ciò, basta creare un workflow custom tramite GUI dalla propria installazione meshroom su pc ed eportarlo in formato .mg in modo poi da copiare il file su google drive ed inserirlo nel comando

*meshroom_compute --forceCompute /content/Mydrive/custom_workflow.mg*





# Come funziona

La logica di questa guida è innanzitutto capire che non si tratta di una semplice pagina web statica ma di una vera e propria guida StepByStep in cui dopo ogni spiegazione è presente direttamente il codice già compilato da eseguire. In ogni cella è già scritto tutto il codice necessario per una elaborazione standard. Il workflow di default che viene eseguito in questa guida consente di leggere una dataset di foto e generare un file obj 3d con le sue relative texture.
il risutato finale sarà:

*   1 file in formato .obj
*   vari file di texture in formato .jpeg
*   1 file con la libreria dei materia in formato .mtl

Il codice presente è modificabile a seconda delle proprie esigenze. Ad esempio se si dispone di un workflow meshroom custom (.mg file)si può eseguire la realtiva cella avendo precedentemente importato il file e avendo l'accortezza di modificare al suo interno tutti i path.

Di default il comando nella cella è commentato, per eseguirlo deve essere decommentato






# Prerequisiti

Lo script per essere eseguito prevede una minima configurazione iniziale e non necessita successivamente di scrivere codice.
Come prerequisito serve avere un'account google drive con spazio a sufficienza per fare l'upload delle foto e salvare la mesh finale. Lo script infatti utilizza come input e output due folder di google drive, le quali dovranno essere create a mano per consentire al codice di leggere le foto, elaborarle e salvare la mesh finale.

# Guida StepByStep

Di seguito uno dopo l'altro sono riportati tutti i passaggi da eseguire che guideranno l'utente nell'elaborazione fotogrammetrica.

Per eseguire il codice basta cliccare sul tasto play (run cell).
Comparirà un messaggio di avviso, andare avanti e cliccare su run anyway.

Per funzionare richiede l'accesso con un account google.

**0. Configurare il runtime in modalità GPU ed eseguire il check**

Questo blocco una volta eseguito consente di verificare se il runtime è configurato con l'utilizzo della GPU. Nel caso in cui l'esito fosse negativo, si riporta di seguito come abilitare tale funzione. Si ricorda che la GPU è fondamentale per una esecuzione più veloce dello script.

*Abilitare le GPU per il notebook:
Andare su Runtime → Change runtime type → Hardware Accelertor → select GPU

---

L' output dello script deve dare come risultato:

**Found GPU at: /device:GPU:0**

in questo caso sarà configurato in modo corretto



In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


**1. Connettere un account Google Drive**

Come prima cosa bisogna aver creato nel filesystem di google drive nella root, due folder chiamate /input e /output


*   dentro /input effettuare l'upload del dataset di foto
*   dentro /output a fine ricostruzione verrà salvata la mesh + textures + mtl

Eseguendo il codice verrà richiesto l'authorization code per autorizzare il runtime ad accedere al filesystem di google drive. Cliccare sul link, copiare la key generata incollarla nella text input premere Invio.

Una volta terminata l'esecuzione si vedrà nel filesystem del runtime il mountpont di google drive dentro /content/drive/MyDrive


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**2. Creare le folder necessarie e fare il download di meshroom:**

Questo codice crea due folder necessarie per ospitare l'eseguibile di meshroom zippato e scompattarlo.

*   /content/temp - contiene il pacchetto meshroom scaricato
*   /content/meshroom - contiene tutti gli eseguibili di meshroom scompattati e
pronti per essere eseguiti

La versione utilizzata di Meshroom è la 2021.1.0 for linux.

In [25]:
%cd /content
!mkdir temp
!mkdir meshroom
%cd temp
!wget -N https://github.com/alicevision/Meshroom/archive/refs/tags/v2023.2.0.tar.gz
!tar -xvf v2023.2.0.tar.gz -C ../meshroom

/content
mkdir: cannot create directory ‘temp’: File exists
mkdir: cannot create directory ‘meshroom’: File exists
/content/temp
--2023-09-20 08:55:08--  https://github.com/alicevision/Meshroom/archive/refs/tags/v2023.2.0.tar.gz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/alicevision/Meshroom/tar.gz/refs/tags/v2023.2.0 [following]
--2023-09-20 08:55:08--  https://codeload.github.com/alicevision/Meshroom/tar.gz/refs/tags/v2023.2.0
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘v2023.2.0.tar.gz’

v2023.2.0.tar.gz        [ <=>                ] 572.35K  --.-KB/s    in 0.09s   

Last-modified header missing -- time-s

**2.1 Importare un workflow custom (facoltativo)**

Se si vuole eseguire la ricostruzione utilizzando un workflow custom personalizzato eseguire la seguente cella decommentando(#) la stringa.

Ricordarsi di aver modificato tutti i path all'interno del file .mg e di averlo importato su google drive rinominando il file in *myworkflow.mg*

In [ ]:
# !/content/meshroom/Meshroom-2021.1.0/meshroom_compute --forceCompute /content/drive/MyDrive/Documents/myworkflow.mg

**3. Eseguire Meshroom**

Se i precendenti blocchi sono stati eseguiti senza errori, si può avviare la cella che effettua la vera e propria ricostruzione fotogrammetrica. Il workflow di default salverà ogni output parziale dentro la folder:

* /tmp/MeshroomCache

l'output finale, come riportato precedentemente, sarà salvato dentro:

* /content/drive/MyDrive/output

---
---

**NOTA BENE: IL RUNTIME HA UN LIMITE DI ESECUZIONE DI 12 ORE, PASSATE LE QUALI VIENE SMONTATO IL FILESYSTEM DI GOOGLE DRIVE E CANCELLATO IL CONTENUTO DI /TMP E /CONTENT. L'ESECUZIONE DEL RUNTIME E' IN BACKGROUND, MA PER VERIFICARE LO STATO DI AVANZAMENTO NON CHIUDERE LA PAGINA WEB.**


In [26]:
!python /content/meshroom/Meshroom-2023.2.0/bin/meshroom_batch --input '/content/drive/MyDrive/input' --output '/content/drive/MyDrive/output'

Traceback (most recent call last):
  File "/content/meshroom/Meshroom-2023.2.0/bin/meshroom_batch", line 7, in <module>
    import meshroom
ModuleNotFoundError: No module named 'meshroom'


# Preview della Mesh (facoltativo)

Una volta terminata la ricostruzione è possibile eseguire una preview della mesh 3d per vierificare che l'elaborazione abbia creato un risultato consistente.

Non è oggetto di questa guida tutto ciò che riguarda il lavoro di postproduzione della mesh.

**0. Preparazione dell'ambiente**

Installare le librerie python numpy e trimesh per eseguire la preview

In [ ]:
!pip install numpy
!pip install trimesh
%cd /content/drive/MyDrive/output

**1. Visualizzazione della mesh**

Importare i moduli numpy e trimesh ed eseguirli

Viene visualizzata una preview della mesh obj per verificare che effettivamente la ricostruzione abbia generato un risultato consistente.

In [ ]:
import numpy as np
import trimesh
mesh = trimesh.load_mesh('texturedMesh.obj')
mesh.show()

*version 1.8*

*Author: jacopo pica*

*jpo85@hotmail.it*

*https://it.linkedin.com/in/jacopo-pica-8164a4161*

*https://github.com/jpo85*
